# IPO Performance
- Retrieves historical price performance
- Uses Yahoo Finance for historical price data
 - https://finance.yahoo.com/

In [1]:
import numpy as np
import pandas as pd

import nasdaq

from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

%matplotlib inline
import matplotlib.pyplot as plt
import time

In [2]:
# params
start_date = '2000-01-01'
ipo_performance_csv = 'IPO Performance.csv'

# Create NASDAQ IPO List

In [3]:
df_symbols = nasdaq.get_ipo_list(start_date)
print('symbols', df_symbols.shape[0])

date range: odict_keys(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10', '2000-11', '2000-12', '2001-01', '2001-02', '2001-03', '2001-04', '2001-05', '2001-06', '2001-07', '2001-08', '2001-09', '2001-10', '2001-11', '2001-12', '2002-01', '2002-02', '2002-03', '2002-04', '2002-05', '2002-06', '2002-07', '2002-08', '2002-09', '2002-10', '2002-11', '2002-12', '2003-01', '2003-02', '2003-03', '2003-04', '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', '2003-10', '2003-11', '2003-12', '2004-01', '2004-02', '2004-03', '2004-04', '2004-05', '2004-06', '2004-07', '2004-08', '2004-09', '2004-10', '2004-11', '2004-12', '2005-01', '2005-02', '2005-03', '2005-04', '2005-05', '2005-06', '2005-07', '2005-08', '2005-09', '2005-10', '2005-11', '2005-12', '2006-01', '2006-02', '2006-03', '2006-04', '2006-05', '2006-06', '2006-07', '2006-08', '2006-09', '2006-10', '2006-11', '2006-12', '2007-01', '2007-02', '2007-03', '2007-04', '2007-05

C:\Users\Void\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
C:\Users\Void\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


symbols 3665


In [4]:
df_symbols.head()

,Company Name,Date Priced,Market,Offer Amount,Price,Shares,Symbol,0,1,2
Symbol,,,,,,,,,,
APL,TARGA PIPELINE PARTNERS LP,2000-01-28,American Stock Exchange,19500000.0,13.0,1.5e+06,APL,NaN,NaN,NaN
ASPM,ASPECT MEDICAL SYSTEMS INC,2000-01-28,NASDAQ,52500000.0,15.0,3.5e+06,ASPM,NaN,NaN,NaN
CAMZ,CAMINUS CORP,2000-01-28,NASDAQ,69952000.0,16.0,4.372e+06,CAMZ,NaN,NaN,NaN
IWAV,INTERWAVE COMMUNICATIONS INTERNATIONAL LTD,2000-01-28,NASDAQ,110500000.0,13.0,8.5e+06,IWAV,NaN,NaN,NaN
LNTY,L90 INC,2000-01-28,NASDAQ,97500000.0,15.0,6.5e+06,LNTY,NaN,NaN,NaN


In [5]:
#load previous to save time
df_prev = pd.read_csv(ipo_performance_csv)
df_prev.index = df_prev['Symbol']

#reduce by previously done
df_symbols = df_symbols.loc[df_symbols.index.difference(df_prev.index)]

# Add Historical Price Performance

- Use open prices only (retial traders won't get best price)

In [6]:
#target variables and trading days range
targets = {
    '1D' : 1,
    '1W' : 5,
    '1M' : 20,
    '3M' : 60
}

In [7]:
#add targets
for t in targets.keys():
    df_symbols[t] = np.NaN

#calculate targets    
counter = 0    
    
for x in df_symbols['Symbol']:
    try:
        time.sleep(5)
        counter += 1
        print('\n( ' + str(counter) + ' / ' + str(df_symbols.shape[0]) + ' ) ' + str(x))

        #get historical prices
        df = pdr.get_data_yahoo(x, start=start_date)

        #add targets
        for t in targets.keys():
            s = df.pct_change(periods=targets[t]).dropna().head(1)['Open']
            if df.head(1).index[0] == df_symbols.at[x, 'Date Priced'] and s.shape[0] > 0 and abs(s[0]) > 0:
                df_symbols.at[x, t] = s[0]
            else:
                df_symbols.at[x, t] = np.NaN
                
        #store
        df_full = pd.concat([df_symbols, df_prev], axis=0)
        df_full.dropna(inplace=True)        
        df_full.to_csv('../Datasets/0 ipo listings.csv', index=False)                
    except Exception as e:
        print(x, e)     


( 1 / 3541 ) AAACU
[*********************100%***********************]  1 of 1 downloaded
AAACU 'Open'

( 2 / 3541 ) AAAP
[*********************100%***********************]  1 of 1 downloaded
AAAP 'Open'

( 3 / 3541 ) AAC
[*********************100%***********************]  1 of 1 downloaded

( 4 / 3541 ) AACC
[*********************100%***********************]  1 of 1 downloaded

( 5 / 3541 ) AACOU
[*********************100%***********************]  1 of 1 downloaded
AACOU 'Open'

( 6 / 3541 ) AACQU
[*********************100%***********************]  1 of 1 downloaded
AACQU 'Open'

( 7 / 3541 ) AAHC
[*********************100%***********************]  1 of 1 downloaded
AAHC 'Open'

( 8 / 3541 ) AAOI
[*********************100%***********************]  1 of 1 downloaded

( 9 / 3541 ) AAPC
[*********************100%***********************]  1 of 1 downloaded
AAPC 'Open'

( 10 / 3541 ) AAT
[*********************100%***********************]  1 of 1 downloaded

( 11 / 3541 ) AATI
[************

[*********************100%***********************]  1 of 1 downloaded
AGIX 'Open'

( 87 / 3541 ) AGLE
[*********************100%***********************]  1 of 1 downloaded
AGLE 'Open'

( 88 / 3541 ) AGMH
[*********************100%***********************]  1 of 1 downloaded
AGMH 'Open'

( 89 / 3541 ) AGNC
[*********************100%***********************]  1 of 1 downloaded
AGNC 'Open'

( 90 / 3541 ) AGNT
[*********************100%***********************]  1 of 1 downloaded
AGNT 'Open'

( 91 / 3541 ) AGO
[*********************100%***********************]  1 of 1 downloaded
AGO 'Open'

( 92 / 3541 ) AGOL
[*********************100%***********************]  1 of 1 downloaded
AGOL 'Open'

( 93 / 3541 ) AGR'A
[*********************100%***********************]  1 of 1 downloaded
AGR'A 'Open'

( 94 / 3541 ) AGRO
[*********************100%***********************]  1 of 1 downloaded
AGRO 'Open'

( 95 / 3541 ) AGRX
[*********************100%***********************]  1 of 1 downloaded
AGRX 'Open'


[*********************100%***********************]  1 of 1 downloaded
AMRH 'Open'

( 171 / 3541 ) AMRS
[*********************100%***********************]  1 of 1 downloaded
AMRS 'Open'

( 172 / 3541 ) AMSF
[*********************100%***********************]  1 of 1 downloaded
AMSF 'Open'

( 173 / 3541 ) AMTG
[*********************100%***********************]  1 of 1 downloaded
AMTG 'Open'

( 174 / 3541 ) AMV
[*********************100%***********************]  1 of 1 downloaded
AMV 'Open'

( 175 / 3541 ) AMWW
[*********************100%***********************]  1 of 1 downloaded
AMWW 'Open'

( 176 / 3541 ) ANAC
[*********************100%***********************]  1 of 1 downloaded
ANAC 'Open'

( 177 / 3541 ) ANDAU
[*********************100%***********************]  1 of 1 downloaded
ANDAU 'Open'

( 178 / 3541 ) ANDS
[*********************100%***********************]  1 of 1 downloaded
ANDS 'Open'

( 179 / 3541 ) ANET
[*********************100%***********************]  1 of 1 downloaded

( 

[*********************100%***********************]  1 of 1 downloaded

( 334 / 3541 ) BARE
[*********************100%***********************]  1 of 1 downloaded
BARE 'Open'

( 335 / 3541 ) BAS
[*********************100%***********************]  1 of 1 downloaded

( 336 / 3541 ) BATT
[*********************100%***********************]  1 of 1 downloaded

( 337 / 3541 ) BAXS
[*********************100%***********************]  1 of 1 downloaded

( 338 / 3541 ) BBBB
[*********************100%***********************]  1 of 1 downloaded
BBBB 'Open'

( 339 / 3541 ) BBEP
[*********************100%***********************]  1 of 1 downloaded
BBEP 'Open'

( 340 / 3541 ) BBG
[*********************100%***********************]  1 of 1 downloaded
BBG 'Open'

( 341 / 3541 ) BBGI
[*********************100%***********************]  1 of 1 downloaded

( 342 / 3541 ) BBJE
[*********************100%***********************]  1 of 1 downloaded
BBJE 'Open'

( 343 / 3541 ) BBND
[*********************100%*******

[*********************100%***********************]  1 of 1 downloaded
CAPN 'Open'

( 501 / 3541 ) CAQCU
[*********************100%***********************]  1 of 1 downloaded
CAQCU 'Open'

( 502 / 3541 ) CARA
[*********************100%***********************]  1 of 1 downloaded

( 503 / 3541 ) CARB
[*********************100%***********************]  1 of 1 downloaded

( 504 / 3541 ) CARE
[*********************100%***********************]  1 of 1 downloaded

( 505 / 3541 ) CASA
[*********************100%***********************]  1 of 1 downloaded

( 506 / 3541 ) CATB
[*********************100%***********************]  1 of 1 downloaded

( 507 / 3541 ) CATM
[*********************100%***********************]  1 of 1 downloaded

( 508 / 3541 ) CATS
[*********************100%***********************]  1 of 1 downloaded

( 509 / 3541 ) CATZ
[*********************100%***********************]  1 of 1 downloaded
CATZ 'Open'

( 510 / 3541 ) CAVM
[*********************100%***********************]  

...

[*********************100%***********************]  1 of 1 downloaded
SCFSP 'Open'

( 2742 / 3541 ) SCGQU
[*********************100%***********************]  1 of 1 downloaded
SCGQU 'Open'

( 2743 / 3541 ) SCMFP
[*********************100%***********************]  1 of 1 downloaded
SCMFP 'Open'

( 2744 / 3541 ) SCMP
[*********************100%***********************]  1 of 1 downloaded

( 2745 / 3541 ) SCOP
[*********************100%***********************]  1 of 1 downloaded
SCOP 'Open'

( 2746 / 3541 ) SCOR
[*********************100%***********************]  1 of 1 downloaded

( 2747 / 3541 ) SCPH
[*********************100%***********************]  1 of 1 downloaded

( 2748 / 3541 ) SCR
[*********************100%***********************]  1 of 1 downloaded
SCR 'Open'

( 2749 / 3541 ) SCRM
[*********************100%***********************]  1 of 1 downloaded
SCRM 'Open'

( 2750 / 3541 ) SCRUF
[*********************100%***********************]  1 of 1 downloaded
SCRUF 'Open'

( 2751 / 354

[*********************100%***********************]  1 of 1 downloaded

( 2907 / 3541 ) SPR
[*********************100%***********************]  1 of 1 downloaded

( 2908 / 3541 ) SPRD
[*********************100%***********************]  1 of 1 downloaded

( 2909 / 3541 ) SPRO
[*********************100%***********************]  1 of 1 downloaded

( 2910 / 3541 ) SPRT
[*********************100%***********************]  1 of 1 downloaded

( 2911 / 3541 ) SPSC
[*********************100%***********************]  1 of 1 downloaded

( 2912 / 3541 ) SPSN
[*********************100%***********************]  1 of 1 downloaded
SPSN 'Open'

( 2913 / 3541 ) SPWH
[*********************100%***********************]  1 of 1 downloaded

( 2914 / 3541 ) SPWR
[*********************100%***********************]  1 of 1 downloaded

( 2915 / 3541 ) SPWX
[*********************100%***********************]  1 of 1 downloaded
SPWX 'Open'

( 2916 / 3541 ) SQ
[*********************100%***********************]  1 of 1 

[*********************100%***********************]  1 of 1 downloaded

( 3075 / 3541 ) TLMR
[*********************100%***********************]  1 of 1 downloaded
TLMR 'Open'

( 3076 / 3541 ) TLND
[*********************100%***********************]  1 of 1 downloaded

( 3077 / 3541 ) TLOG
[*********************100%***********************]  1 of 1 downloaded

( 3078 / 3541 ) TLP
[*********************100%***********************]  1 of 1 downloaded

( 3079 / 3541 ) TLVT
[*********************100%***********************]  1 of 1 downloaded
TLVT 'Open'

( 3080 / 3541 ) TLWT
[*********************100%***********************]  1 of 1 downloaded
TLWT 'Open'

( 3081 / 3541 ) TLXS
[*********************100%***********************]  1 of 1 downloaded
TLXS 'Open'

( 3082 / 3541 ) TLYS
[*********************100%***********************]  1 of 1 downloaded

( 3083 / 3541 ) TMCXU
[*********************100%***********************]  1 of 1 downloaded

( 3084 / 3541 ) TMH
[*********************100%*******

[*********************100%***********************]  1 of 1 downloaded

( 3239 / 3541 ) USBE
[*********************100%***********************]  1 of 1 downloaded
USBE 'Open'

( 3240 / 3541 ) USCI
[*********************100%***********************]  1 of 1 downloaded

( 3241 / 3541 ) USDP
[*********************100%***********************]  1 of 1 downloaded

( 3242 / 3541 ) USFD
[*********************100%***********************]  1 of 1 downloaded

( 3243 / 3541 ) USHP
[*********************100%***********************]  1 of 1 downloaded
USHP 'Open'

( 3244 / 3541 ) USIH
[*********************100%***********************]  1 of 1 downloaded
USIH 'Open'

( 3245 / 3541 ) USL
[*********************100%***********************]  1 of 1 downloaded

( 3246 / 3541 ) USOD
[*********************100%***********************]  1 of 1 downloaded

( 3247 / 3541 ) USOU
[*********************100%***********************]  1 of 1 downloaded

( 3248 / 3541 ) USPI
[*********************100%*******************

[*********************100%***********************]  1 of 1 downloaded
WITS 'Open'

( 3407 / 3541 ) WIX
[*********************100%***********************]  1 of 1 downloaded

( 3408 / 3541 ) WJCI
[*********************100%***********************]  1 of 1 downloaded
WJCI 'Open'

( 3409 / 3541 ) WK
[*********************100%***********************]  1 of 1 downloaded

( 3410 / 3541 ) WLDN
[*********************100%***********************]  1 of 1 downloaded

( 3411 / 3541 ) WLH
[*********************100%***********************]  1 of 1 downloaded

( 3412 / 3541 ) WLK
[*********************100%***********************]  1 of 1 downloaded

( 3413 / 3541 ) WLKP
[*********************100%***********************]  1 of 1 downloaded

( 3414 / 3541 ) WLL
[*********************100%***********************]  1 of 1 downloaded

( 3415 / 3541 ) WLRHU
[*********************100%***********************]  1 of 1 downloaded
WLRHU 'Open'

( 3416 / 3541 ) WLSC
[*********************100%**********************

In [8]:
print('done', df_symbols.shape[0], 'prev', df_prev.shape[0])
df_symbols.head(3)

done 3541 prev 124


,Company Name,Date Priced,Market,Offer Amount,Price,Shares,Symbol,0,1,2,1D,1W,1M,3M
Symbol,,,,,,,,,,,,,,
AAACU,ASIA AUTOMOTIVE ACQUISITION CORP.,2006-04-13,OTCBB,35000000.0,8.0,4.375e+06,AAACU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAAP,ADVANCED ACCELERATOR APPLICATIONS S.A.,2015-11-11,NASDAQ,75008000.0,16.0,4688000,AAAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAC,"AAC HOLDINGS, INC.",2014-10-02,New York Stock Exchange,75000000.0,15.0,5000000,AAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
